# Group existing generators within regions and create new resources

This notebook shows how to use the PowerGenome API to cluster existing resources and create new-build resource options by model region. You'll need a settings file, such as the `test_settings.yml` provided in the `example_system` folder, with the following parameters.

For both existing and new resource:
- model_regions
- region_aggregations
- model_year
- target_usd_year
- atb_usd_year
- startup_fuel_use
- startup_vom_costs_mw
- startup_vom_costs_usd_year
- startup_costs_type
- startup_costs_per_cold_start_mw
- startup_costs_per_cold_start_usd_year
- existing_startup_costs_tech_map
- new_build_startup_costs

Specific to existing resource:
- num_clusters
- retirement_ages
- atb_existing_year
- existing_om_muiltiplier
- eia_atb_tech_map
- proposed_status_included
- proposed_gen_heat_rates
- proposed_min_load

Specific to new-build resources:
- atb_cost_case
- atb_financial_case
- atb_cap_recovery_years
- atb_new_gen
- renewables_clusters
- cost_multiplier_region_map
- cost_multiplier_technology_map
- transmission_investment_cost (if spur-lines are needed for non-renewable resources)

To calculate fuel costs for each resource:
- aeo_fuel_region_map
- eia_series_region_names
- eia_series_fuel_names
- eia_aeo_year
- eia_series_scenario_names
- aeo_fuel_scenarios
- aeo_fuel_usd_year
- tech_fuel_map
- fuel_emission_factors
- carbon_tax (optional)

And if CCS resources are included:
- ccs_fuel_map
- ccs_capture_rate
- ccs_disposal_cost

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import warnings

warnings.simplefilter("ignore")

In [3]:
from pathlib import Path

import geopandas as gpd
import numpy as np
import pandas as pd
from powergenome.generators import GeneratorClusters
from powergenome.GenX import reduce_time_domain
from powergenome.load_profiles import make_final_load_curves
from powergenome.params import DATA_PATHS
from powergenome.util import (
    build_scenario_settings,
    init_pudl_connection,
    load_settings,
)
from powergenome.external_data import (
    make_demand_response_profiles,
    make_generator_variability,
)

pd.options.display.max_columns = 200

## Import settings
This assumes that the settings file is set up for multiple scenarios/planning periods. If you are using a settings file with only a single scenario/planning period, remove or comment out the line with `build_scenario_settings`.

In [7]:
pudl_engine, pudl_out = init_pudl_connection()
cwd = Path.cwd()

settings_path = (
    cwd.parent / "example_system" / "test_settings.yml"
)
settings = load_settings(settings_path)
settings["input_folder"] = settings_path.parent / settings["input_folder"]
scenario_definitions = pd.read_csv(
    settings["input_folder"] / settings["scenario_definitions_fn"]
)
scenario_settings = build_scenario_settings(settings, scenario_definitions)

## Initialize a `GeneratorClusters` object

`GeneratorClusters` is how existing generators are clustered, and it provides a convinience method for creating new-build resources (`GeneratorClusters.create_new_generators`).

In [5]:
gc = GeneratorClusters(pudl_engine, pudl_out, scenario_settings[2030]["p1"])

961.5000000000002  MW without lat/lon


### Existing generators

Lets look at how existing generators have been clustered in the different regions. The raw output from `create_region_technology_clusters` has lots of extra columns that might not be needed. Some of them include:

- `resource` is a snake case version of `technology`.
- `unmodified_cap_size` is the average size of generating units. `Cap_size` is `unmodified_cap_size` multiplied by the capacity factor for technologies that are derated by their CF.
- `Existing_Cap_MW` is `Cap_size` multiplied by `num_units`.
- `heat_rate_mmbtu_mwh_iqr` and `heat_rate_mmbtu_mwh_std` are measurements of how widely the heate rate varies within a cluster. If these values are large compared to `Heat_rate_MMBTU_per_MWh` then you might consider increasing the number of clusters for that technology/region.
- `Fuel` is a combination of the region (assigned by the settings parameter `fuel_region_map`), the AEO scenario (assigned by the settings parameter `aeo_fuel_scenarios`), and the fuel type for that generator resource (assigned by the settings parameters `tech_fuel_map` and `ccs_fuel_map`). Only fuels in AEO are included, which means that biomass/hydrogen/RNG, etc are not options.

The various "model tags" that are included in the settings file are unique to GenX. If you find them useful, they can be used to assign various values of any data type to generators in a column name of your choosing. If you don't want to use them, they can be ignored or removed via `settings.pop(<name>)`.

In [ ]:
# gc.settings.pop("model_tag_names")
# gc.settings.pop("model_tag_values")
# gc.settings.pop("regional_tag_values")

In [12]:
existing_gen = gc.create_region_technology_clusters()

Creating gdf
['Solar Photovoltaic', 'Onshore Wind Turbine', 'Batteries', 'Biomass', 'Conventional Hydroelectric', 'Natural Gas Fired Combined Cycle', 'Other_peaker', 'All Other', 'Natural Gas Fired Combustion Turbine', 'Other Natural Gas']


No model tag values found for DR ('NoneType' object has no attribute 'items')


In [13]:
existing_gen

,region,technology,cluster,index,Cap_size,minimum_load_mw,Heat_rate_MMBTU_per_MWh,Fixed_OM_cost_per_MWyr,Var_OM_cost_per_MWh,heat_rate_mmbtu_mwh_iqr,heat_rate_mmbtu_mwh_std,Min_power,num_units,capacity_factor,unmodified_cap_size,Existing_Cap_MW,unmodified_existing_cap_mw,Start_fuel_MMBTU_per_MW,Fuel,Start_cost_per_MW,THERM,DISP,NDISP,STOR,DR,HYDRO,Commit,RPS,CES,New_Build,Hydro_level,Resource,profile
0,CA_N,Biomass,1,0,0.91,0.973,17.202,111708.000,7.106,1.484,4.645,0.374,81,0.350,2.602,73.71,210.762,0.0,None,0.0,1,0,1,0,0,0,1,1,1,-1,0.0,biomass,None
1,CA_N,Conventional Hydroelectric,1,1,62.17,3.740,9.118,44560.000,0.000,0.000,0.009,0.060,117,0.308,62.171,7273.89,7274.007,0.0,None,0.0,0,0,0,0,0,1,0,0,1,-1,0.5,conventional_hydroelectric,"[0.21636393788244979, 0.21612066334138946, 0.2..."
2,CA_N,Geothermal,1,2,27.95,14.878,9.118,198040.000,0.000,0.000,0.000,0.389,23,0.730,38.278,642.85,880.394,0.0,None,0.0,1,0,1,0,0,0,0,1,1,-1,0.0,geothermal,None
3,CA_N,Hydroelectric Pumped Storage,1,3,89.55,23.850,0.000,38460.000,0.000,0.000,0.000,0.266,20,-0.022,89.550,1791.00,1791.000,0.0,None,0.0,0,0,0,1,0,0,0,0,0,0,0.0,hydroelectric_pumped_storage,None
4,CA_N,Natural Gas Fired Combined Cycle,1,4,309.16,148.158,7.476,11179.164,3.619,1.416,1.473,0.479,33,0.374,309.161,10202.28,10202.313,2.0,pacific_reference_naturalgas,95.0,1,0,0,0,0,0,1,0,0,1,0.0,natural_gas_fired_combined_cycle,None
5,CA_N,Natural Gas Fired Combustion Turbine,1,5,57.27,29.941,10.693,9193.048,10.800,2.225,1.993,0.523,59,0.149,57.266,3378.93,3378.694,3.5,pacific_reference_naturalgas,123.0,1,0,0,0,0,0,1,0,0,1,0.0,natural_gas_fired_combustion_turbine,None
6,CA_N,Onshore Wind Turbine,1,6,47.66,4.307,9.118,43205.000,0.000,0.000,0.024,0.090,28,0.289,47.657,1327.60,1334.396,0.0,None,0.0,0,1,0,0,0,0,0,1,1,0,0.0,onshore_wind_turbine,"[0.0004, 0.0293, 0.0617, 0.0811, 0.0819, 0.015..."
7,CA_N,Small Hydroelectric,1,7,1.81,0.468,9.118,44560.000,0.000,0.000,0.002,0.085,133,0.330,5.484,240.73,729.372,0.0,None,0.0,0,0,1,0,0,0,0,1,1,-1,0.0,small_hydroelectric,"[0.21636393788244979, 0.21612066334138946, 0.2..."
8,CA_N,Solar Photovoltaic,1,8,17.27,0.000,9.134,17943.000,0.000,0.000,0.022,0.000,190,0.217,17.275,2787.10,3282.250,0.0,None,0.0,0,1,0,0,0,0,0,1,1,0,0.0,solar_photovoltaic,"[0.2899, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0...."
9,CA_S,Biomass,1,9,1.70,2.190,12.443,111708.000,5.129,1.510,2.760,0.584,62,0.453,3.753,105.40,232.686,0.0,None,0.0,1,0,1,0,0,0,1,1,1,-1,0.0,biomass,None


### New generators

New generators are based on data from NREL's Annual Technology Baseline (ATB). ATB uses a format of `\<technology>_<tech_detail>` to describe resources, such as `NaturalGas_CTAvgCF`. There are additional parameters including the cost case and financial case that are needed to specify the capex for a specific generator type. Note that many of the `tech_detail` values have identical capex and O&M values, so it doesn't matter which one is used. Examples include the capacity factor of combustion resources (`CTAvgCF` vs `CTHighCF`) and UtilityPV (`Chicago`, `KansasCity`, and `LosAngeles` have different capacity factors that we don't use in PowerGenome).

The raw output from `create_new_generators` has many more columns than existing generators. Several of these - such as `basis_year`, `capex`, `capex_mwh`, `cap_recovery_years`, `waccnomtech`, `regional_cost_multiplier`, and `interconnect_annuity` - are used to calculate the final `Inv_cost_per_MWyr` and `Inv_cost_per_MWhyr`. `lcoe` is pre-calculated using 2030 mid-range ATB costs, and is not specific to the model year for each run. The underlying data are retained here so they can be easily reviewed, but only keep the columns that you want/need.

The column `variability` has array values with the annual generation profiles for a resource. If these profiles represent 2012 they have 8784 hourly values.

The column `Max_Cap_MW` has a value of -1 if there is no limit on the capacity. 

If the user has included demand response profiles as an external file (using the `demand_response_fn` parameter in settings and the associated `demand_response` and `demand_response_resources` parameters), demand response resources will be included.

### Modified and custom new generators

By default, new generators are created using data from NREL ATB. But it is possible to modify an existing ATB generator type or create a modified copy of an ATB generator. To understand 

Modified ATB generators are included in the `atb_modifiers` parameter. In ATB 2019, NREL used a blend of combustion turbine and combined cycle types that we felt was unrealistic. Therefore we have included, by default, the following modifications. The modifying values are provided as `[<op>, <value>]`, where `op` is an operator of type `add`, `mul`, `sub`, and `truediv`. Again, this parameter modifies an existing technology in-place.

```
atb_modifiers:
  ngct:
    technology: NaturalGas
    tech_detail: CTAvgCF
    capex: [mul, 0.76]
    Var_OM_cost_per_MWh: [mul, 1.51]
    Fixed_OM_cost_per_MWyr: [mul, 0.56]
    Heat_rate_MMBTU_per_MWh: [mul, 0.97]
  ngcc:
    technology: NaturalGas
    tech_detail: CCAvgCF
    capex: [mul, 0.89]
    Var_OM_cost_per_MWh: [mul, 0.73]
    Fixed_OM_cost_per_MWyr: [mul, 0.95]
    Heat_rate_MMBTU_per_MWh: [mul, 0.98]
```

New generators based on modifications of an existing ATB generator are included in the parameter `modified_atb_new_gen`. The example below creates a NGCC resource called `NaturalGas_CCS100_Mid` with 100% CO₂ capture. Any new resources need to be added to the parameters `cost_multiplier_technology_map`, `new_build_startup_costs`, and `model_tag_values` (optional). Names are linked using string matching, so the full name doesn't need to be included in these parameters (but it should be long enough to be unique).

```
modified_atb_new_gen:
  NGCCS100:
    new_technology: NaturalGas
    new_tech_detail: CCS100
    new_cost_case: Mid
    atb_technology: NaturalGas
    atb_tech_detail: CCCCSAvgCF
    atb_cost_case: Mid
    size_mw: 500
    capex: [add, 116000]
    heat_rate: [add, 0.365]
    o_m_fixed_mw: [add, 9670]
    o_m_variable_mwh: [mul, 1.076]
```

### New wind/solar resources

In the settings, uses can specify the type of resource, maximum capacity, number of clusters, and maximum LCOE (optional) in a model region. With these parameters, PowerGenome combines pre-clustered groups of resources, calculating the total capacity and a weighted generation profile for each resource. Utility PV capacity in these clusters is calculated at 45 MW/km^2, and we recommend that users deflate this value (`cap_multiplier`). For offshore wind, specify fixed/floating (`turbine_type`) and if the sites are part of BOEM lease areas (`pref_site`).

```
renewables_clusters:
  - region: CA_N
    technology: landbasedwind
    max_clusters: 2
    max_lcoe: 110
    min_capacity: 25000
  - region: CA_N
    technology: offshorewind
    turbine_type: floating
    pref_site: 1
    max_clusters: 3
    min_capacity: 40000
  - region: CA_S
    technology: landbasedwind
    max_clusters: 4
    max_lcoe: 100
    min_capacity: 45000
  - region: CA_S
    technology: utilitypv
    max_clusters: 5
    max_lcoe: 75
    min_capacity: 100000
    cap_multiplier: 0.2
  - region: WECC_AZ
    technology: utilitypv
    max_clusters: 3
    min_capacity: 100000
    cap_multiplier: 0.2
```

**NOTE:** Wind/solar resources have a value in the `LCOE` column. This is a pre-computed LCOE that assumes 2030 mid-range capex values from ATB 2019, and is not specific to settings file you are using. This LCOE is used to cluster project areas using a hierarchical clustering method. 

### Interconnection costs
Wind and solar resources have pre-calculated interconnection costs, so the user doesn't need to supply any additional data. But if a spur line is needed for thermal resources, these distances should be included under the column `spur_miles` in the file specified by `capacity_limit_spur_fn`. This spur line distance is then multiplied by costs for each model region listed in the `transmission_investment_cost` parameter. Spur line capex in the example settings file is from ReEDS documentation. You can look at [a mapping of IPM regions to ReEDS regions](https://github.com/gschivley/pg_misc/blob/master/create_clusters/site_interconnection_costs.py#L32-L155) and the associated transmission costs that I have compiled.

In [9]:
new_gen = gc.create_new_generators()

Selected technology landbasedwind capacity in region CA_N less than minimum (8424.4314 < 25000 MW)
Selected technology landbasedwind capacity in region CA_S less than minimum (23639.682500000003 < 45000 MW)
No model tag values found for DR ('NoneType' object has no attribute 'items')
Transmission investment costs are missing or zero for some resources and will not be included in the total investment costs.


In [10]:
new_gen.columns

Index(['technology', 'basis_year', 'Fixed_OM_cost_per_MWyr',
       'Fixed_OM_cost_per_MWhyr', 'Var_OM_cost_per_MWh', 'capex', 'capex_mwh',
       'Inv_cost_per_MWyr', 'Inv_cost_per_MWhyr', 'Heat_rate_MMBTU_per_MWh',
       'Cap_size', 'cap_recovery_years', 'waccnomtech', 'Max_Cap_MW', 'region',
       'regional_cost_multiplier', 'ids', 'area', 'm_popden', 'metro_id',
       'ipm_region', 'site_substation_spur_miles', 'substation_metro_tx_miles',
       'site_metro_spur_miles', 'interconnect_annuity', 'lcoe', 'spur_miles',
       'tx_miles', 'profile', 'existing', 'cluster', 'offshore_spur_miles',
       'turbine_type', 'pref_site', 'Start_fuel_MMBTU_per_MW', 'Fuel',
       'Start_cost_per_MW', 'THERM', 'DISP', 'NDISP', 'STOR', 'DR', 'HYDRO',
       'Commit', 'RPS', 'CES', 'New_Build', 'Hydro_level', 'spur_capex',
       'spur_inv_mwyr', 'offshore_spur_capex', 'offshore_spur_inv_mwyr',
       'tx_capex', 'tx_inv_mwyr', 'plant_inv_cost_mwyr', 'R_ID', 'zone',
       'voltage_level', 'Cap

#### Spur line/interconnection distances and capacity limits
Interconnection distances and the maximum available capacity for UtilityPV, LandbasedWind, and OffshoreWind are all included in the data used to select and combine clusters. Spur line distances for other resources must be provided by the user in a CSV file and included in the settings file as `capacity_limit_spur_fn`.

In [17]:
cols = [
    "region",
    "technology",
    "cluster",
    "Max_Cap_MW",
    "lcoe",
    "regional_cost_multiplier",
    "Inv_cost_per_MWyr",
    "plant_inv_cost_mwyr",
    "Start_cost_per_MW",
    "interconnect_annuity",
    "spur_inv_mwyr",
    "spur_miles",
    "offshore_spur_inv_mwyr",
    "tx_inv_mwyr",
    "profile",
]
new_gen[cols]

,region,technology,cluster,Max_Cap_MW,lcoe,regional_cost_multiplier,Inv_cost_per_MWyr,plant_inv_cost_mwyr,Start_cost_per_MW,interconnect_annuity,spur_inv_mwyr,spur_miles,offshore_spur_inv_mwyr,tx_inv_mwyr,profile
0,CA_N,NaturalGas_CCCCSAvgCF_Mid,0.0,-1.00000,0.000000,1.090697,234735.910407,221996.0,87.0,0.000000,12739.910407,20.000000,0.000000,0.0,0
1,CA_N,NaturalGas_CCAvgCF_Mid,0.0,-1.00000,0.000000,1.339623,117344.000000,117344.0,87.0,0.000000,0.000000,0.000000,0.000000,0.0,0
2,CA_N,NaturalGas_CTAvgCF_Mid,0.0,-1.00000,0.000000,1.211268,91163.000000,91163.0,113.0,0.000000,0.000000,0.000000,0.000000,0.0,0
3,CA_N,Battery_*_Mid,0.0,-1.00000,0.000000,1.041215,54124.000000,54124.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0
4,CA_N,Nuclear_mid__,0.0,-1.00000,0.000000,1.249179,612569.776018,580720.0,235.0,0.000000,31849.776018,50.000000,0.000000,0.0,0
5,CA_N,NaturalGas_CCS100_Mid,0.0,-1.00000,0.000000,1.090697,247089.910407,234350.0,87.0,0.000000,12739.910407,20.000000,0.000000,0.0,0
6,CA_N,LandbasedWind_LTRG1_Mid,1.0,1259.55000,109.317400,2.109174,326281.493100,288184.0,0.0,38097.493100,7928.683242,12.447000,0.000000,0.0,"[0.08843548, 0.22878414, 0.25738907, 0.3396663..."
7,CA_N,LandbasedWind_LTRG1_Mid,2.0,7164.88140,101.403571,2.109174,324869.661403,288184.0,0.0,36685.661403,19510.610728,30.629118,0.000000,0.0,"[0.034569643, 0.03650082, 0.055118807, 0.05934..."
8,CA_N,OffShoreWind_OTRG10_Mid,1.0,18920.54330,161.759154,1.308483,939767.255090,476591.0,0.0,463176.255090,138210.374951,216.972287,177276.430331,0.0,"[0.05657291, 0.028173681, 0.09259045, 0.070540..."
9,CA_N,OffShoreWind_OTRG10_Mid,2.0,15748.00000,172.627669,1.308483,856025.037885,476591.0,0.0,379434.037885,45103.736382,70.806991,229905.779571,0.0,"[0.34054095, 0.4315498, 0.3708048, 0.34306705,..."


#### Generation profiles

Hourly generation profiles are saved in a `variability` column of the dataframe. These are then extracted using the function `make_generator_variability`. The variability (generation profile) dataframe is in the same (column) order as rows in the generator dataframe.

In [14]:
existing_variability = make_generator_variability(existing_gen)
existing_variability

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26
0,1.0,0.216364,1.0,1.0,1.0,1.0,0.0004,0.216364,0.2899,1.0,0.105270,1.0,1.0,1.0,1.0,0.016339,0.105270,0.282178,1.0,0.2591,1.0,1.0,1.0,1.0,1.0,0.1325,0.2031
1,1.0,0.216121,1.0,1.0,1.0,1.0,0.0293,0.216121,0.0000,1.0,0.105162,1.0,1.0,1.0,1.0,0.112207,0.105162,0.000000,1.0,0.2593,1.0,1.0,1.0,1.0,1.0,0.2742,0.0000
2,1.0,0.215877,1.0,1.0,1.0,1.0,0.0617,0.215877,0.0000,1.0,0.105108,1.0,1.0,1.0,1.0,0.302212,0.105108,0.000000,1.0,0.2595,1.0,1.0,1.0,1.0,1.0,0.5171,0.0000
3,1.0,0.215634,1.0,1.0,1.0,1.0,0.0811,0.215634,0.0000,1.0,0.105053,1.0,1.0,1.0,1.0,0.506039,0.105053,0.000000,1.0,0.2597,1.0,1.0,1.0,1.0,1.0,0.3361,0.0000
4,1.0,0.215384,1.0,1.0,1.0,1.0,0.0819,0.215384,0.0000,1.0,0.104999,1.0,1.0,1.0,1.0,0.476053,0.104999,0.000000,1.0,0.2599,1.0,1.0,1.0,1.0,1.0,0.3464,0.0000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8755,1.0,0.217673,1.0,1.0,1.0,1.0,0.0204,0.217673,0.6897,1.0,0.105595,1.0,1.0,1.0,1.0,0.094797,0.105595,0.689717,1.0,0.2581,1.0,1.0,1.0,1.0,1.0,0.0300,0.6802
8756,1.0,0.217430,1.0,1.0,1.0,1.0,0.0000,0.217430,0.7108,1.0,0.105541,1.0,1.0,1.0,1.0,0.076850,0.105541,0.731451,1.0,0.2583,1.0,1.0,1.0,1.0,1.0,0.0198,0.6798
8757,1.0,0.217187,1.0,1.0,1.0,1.0,0.0000,0.217187,0.6974,1.0,0.105433,1.0,1.0,1.0,1.0,0.021176,0.105433,0.730564,1.0,0.2585,1.0,1.0,1.0,1.0,1.0,0.0366,0.6706
8758,1.0,0.216943,1.0,1.0,1.0,1.0,0.0000,0.216943,0.6187,1.0,0.105377,1.0,1.0,1.0,1.0,0.003680,0.105377,0.640714,1.0,0.2587,1.0,1.0,1.0,1.0,1.0,0.0536,0.5739


Since the variability column names are only integers, it can help to replace them with descriptive strings.

In [15]:
existing_variability.columns = (
    existing_gen["region"]
    + "_"
    + existing_gen["Resource"]
    + "_"
    + existing_gen["cluster"].astype(str)
)
existing_variability

,CA_N_biomass_1,CA_N_conventional_hydroelectric_1,CA_N_geothermal_1,CA_N_hydroelectric_pumped_storage_1,CA_N_natural_gas_fired_combined_cycle_1,CA_N_natural_gas_fired_combustion_turbine_1,CA_N_onshore_wind_turbine_1,CA_N_small_hydroelectric_1,CA_N_solar_photovoltaic_1,CA_S_biomass_1,CA_S_conventional_hydroelectric_1,CA_S_geothermal_1,CA_S_hydroelectric_pumped_storage_1,CA_S_natural_gas_fired_combined_cycle_1,CA_S_natural_gas_fired_combustion_turbine_1,CA_S_onshore_wind_turbine_1,CA_S_small_hydroelectric_1,CA_S_solar_photovoltaic_1,WECC_AZ_biomass_1,WECC_AZ_conventional_hydroelectric_1,WECC_AZ_conventional_steam_coal_1,WECC_AZ_hydroelectric_pumped_storage_1,WECC_AZ_natural_gas_fired_combined_cycle_1,WECC_AZ_natural_gas_fired_combustion_turbine_1,WECC_AZ_nuclear_1,WECC_AZ_onshore_wind_turbine_1,WECC_AZ_solar_photovoltaic_1
0,1.0,0.216364,1.0,1.0,1.0,1.0,0.0004,0.216364,0.2899,1.0,0.105270,1.0,1.0,1.0,1.0,0.016339,0.105270,0.282178,1.0,0.2591,1.0,1.0,1.0,1.0,1.0,0.1325,0.2031
1,1.0,0.216121,1.0,1.0,1.0,1.0,0.0293,0.216121,0.0000,1.0,0.105162,1.0,1.0,1.0,1.0,0.112207,0.105162,0.000000,1.0,0.2593,1.0,1.0,1.0,1.0,1.0,0.2742,0.0000
2,1.0,0.215877,1.0,1.0,1.0,1.0,0.0617,0.215877,0.0000,1.0,0.105108,1.0,1.0,1.0,1.0,0.302212,0.105108,0.000000,1.0,0.2595,1.0,1.0,1.0,1.0,1.0,0.5171,0.0000
3,1.0,0.215634,1.0,1.0,1.0,1.0,0.0811,0.215634,0.0000,1.0,0.105053,1.0,1.0,1.0,1.0,0.506039,0.105053,0.000000,1.0,0.2597,1.0,1.0,1.0,1.0,1.0,0.3361,0.0000
4,1.0,0.215384,1.0,1.0,1.0,1.0,0.0819,0.215384,0.0000,1.0,0.104999,1.0,1.0,1.0,1.0,0.476053,0.104999,0.000000,1.0,0.2599,1.0,1.0,1.0,1.0,1.0,0.3464,0.0000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8755,1.0,0.217673,1.0,1.0,1.0,1.0,0.0204,0.217673,0.6897,1.0,0.105595,1.0,1.0,1.0,1.0,0.094797,0.105595,0.689717,1.0,0.2581,1.0,1.0,1.0,1.0,1.0,0.0300,0.6802
8756,1.0,0.217430,1.0,1.0,1.0,1.0,0.0000,0.217430,0.7108,1.0,0.105541,1.0,1.0,1.0,1.0,0.076850,0.105541,0.731451,1.0,0.2583,1.0,1.0,1.0,1.0,1.0,0.0198,0.6798
8757,1.0,0.217187,1.0,1.0,1.0,1.0,0.0000,0.217187,0.6974,1.0,0.105433,1.0,1.0,1.0,1.0,0.021176,0.105433,0.730564,1.0,0.2585,1.0,1.0,1.0,1.0,1.0,0.0366,0.6706
8758,1.0,0.216943,1.0,1.0,1.0,1.0,0.0000,0.216943,0.6187,1.0,0.105377,1.0,1.0,1.0,1.0,0.003680,0.105377,0.640714,1.0,0.2587,1.0,1.0,1.0,1.0,1.0,0.0536,0.5739


In [16]:
make_generator_variability(new_gen)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36
0,1.0,1.0,1.0,1.0,1.0,1.0,0.088435,0.034570,0.056573,0.340541,0.053322,1.0,1.0,1.0,1.0,1.0,1.0,0.064086,0.072826,0.049048,0.070055,0.381206,0.354309,0.392327,0.430983,0.380811,1.0,1.0,1.0,1.0,1.0,1.0,0.307749,0.295572,0.295854,0.089249,0.089300
1,1.0,1.0,1.0,1.0,1.0,1.0,0.228784,0.036501,0.028174,0.431550,0.003695,1.0,1.0,1.0,1.0,1.0,1.0,0.148605,0.118585,0.091472,0.130442,0.000000,0.000000,0.000000,0.000000,0.000000,1.0,1.0,1.0,1.0,1.0,1.0,0.000000,0.000000,0.000000,0.083333,0.083399
2,1.0,1.0,1.0,1.0,1.0,1.0,0.257389,0.055119,0.092590,0.370805,0.042638,1.0,1.0,1.0,1.0,1.0,1.0,0.319481,0.082894,0.122291,0.179688,0.000000,0.000000,0.000000,0.000000,0.000000,1.0,1.0,1.0,1.0,1.0,1.0,0.000000,0.000000,0.000000,0.032116,0.032127
3,1.0,1.0,1.0,1.0,1.0,1.0,0.339666,0.059342,0.070540,0.343067,0.056868,1.0,1.0,1.0,1.0,1.0,1.0,0.474706,0.134995,0.281698,0.272368,0.000000,0.000000,0.000000,0.000000,0.000000,1.0,1.0,1.0,1.0,1.0,1.0,0.000000,0.000000,0.000000,0.024341,0.024390
4,1.0,1.0,1.0,1.0,1.0,1.0,0.410760,0.082231,0.018969,0.323002,0.012820,1.0,1.0,1.0,1.0,1.0,1.0,0.474848,0.210213,0.353403,0.351928,0.000000,0.000000,0.000000,0.000000,0.000000,1.0,1.0,1.0,1.0,1.0,1.0,0.000000,0.000000,0.000000,0.015382,0.015473
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8755,1.0,1.0,1.0,1.0,1.0,1.0,0.098973,0.355000,0.007967,0.154682,0.000000,1.0,1.0,1.0,1.0,1.0,1.0,0.237970,0.029499,0.121557,0.076308,0.906424,0.931416,0.875831,0.940755,0.920576,1.0,1.0,1.0,1.0,1.0,1.0,0.860794,0.865946,0.876416,0.621535,0.621689
8756,1.0,1.0,1.0,1.0,1.0,1.0,0.116479,0.311046,0.004159,0.037722,0.000000,1.0,1.0,1.0,1.0,1.0,1.0,0.181572,0.045488,0.125199,0.081166,0.930923,0.944681,0.940882,0.957254,0.955774,1.0,1.0,1.0,1.0,1.0,1.0,0.890613,0.871960,0.881094,0.604462,0.604511
8757,1.0,1.0,1.0,1.0,1.0,1.0,0.021648,0.264090,0.000000,0.008297,0.000000,1.0,1.0,1.0,1.0,1.0,1.0,0.087203,0.010657,0.065488,0.049913,0.952745,0.932476,0.911772,0.945872,0.949151,1.0,1.0,1.0,1.0,1.0,1.0,0.847878,0.843627,0.858192,0.511832,0.511802
8758,1.0,1.0,1.0,1.0,1.0,1.0,0.011410,0.210792,0.000000,0.004214,0.000000,1.0,1.0,1.0,1.0,1.0,1.0,0.039239,0.000517,0.007709,0.040333,0.857150,0.843580,0.821995,0.850169,0.841343,1.0,1.0,1.0,1.0,1.0,1.0,0.744098,0.741382,0.758446,0.089249,0.089300
